# 1. 参数的更新

神经网络的学习的目的是找到使损失函数的值尽可能小的参数。这是寻找最优参数的问题，解决这个问题的过程称为最优化（optimization）。遗憾的是，神经网络的最优化问题非常难。这是因为参数空间非常复杂，无法轻易找到最优解。而且，在深度神经网络中，参数的数量非常庞大，导致最优化问题更加复杂。

## 1.1 SGD

随机梯度下降法（Stochastic Gradient Descent，SGD）是一个简单的方法，不过比起胡乱地搜索参数空间，也算是“聪明”的方法。

$$ W = W - \eta \frac{\partial L}{\partial W} $$

这里把需要更新的权重参数记为 $W$，把损失函数关于 $W$ 的梯度记为 $\frac{\partial L}{\partial W}$。$\eta$ 表示学习率，实际计算中会取 0.01 或0.001 这些事先决定好的值。现在，我们将 SGD 算法实现为一个名为 SGD 的 Python 类，

In [2]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

进行初始化时的参数 lr 表示学习率（learning rate）。代码段中还定义了update(params, grads)方法，这个方法在SGD中会被反复调用。参数params和grads（与之前的神经网络的实现一样）是字典型变量，按params['W1']、grads['W1']的形式，分别保存了权重参数和它们的梯度。

可以按如下方式进行神经网络的参数的更新：

```python
network = TwoLayerNet(...)
optimizer = SGD()    # 优化器对象

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...) # mini-batch
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...
```

虽然SGD简单容易实现，但是在解决某些问题时可能没有效率。这里思考一下求下面这个函数的最小值的问题。

$$ f(x, y) = \frac{1}{20}x^2 + y^2 $$

这个函数梯度的特征是，y 轴方向上大，x 轴方向上小。我们尝试对该函数应用 SGD，从 $(x,y)=(-7, 2)$ 初始点开始搜索，如下图所示，搜索路径呈“之”字形移动。

![img](images/chapter12/SGD.png)

SGD低效的根本原因是，梯度的方向并没有指向最小值的方向。为了改正 SGD 的缺点，下面我们将介绍 Momentum、AdaGrad、Adam 这三种方法来取代SGD。

## 1.2 Momentum

Momentum是“动量”的意思，数学式表示Momentum方法如下：
$$ v = \alpha v - \eta \frac{\partial L}{\partial W} $$
$$ W = W + v $$